In [1]:
import math
import xarray as xr
import cartopy.crs as crs
import matplotlib as mpl
from matplotlib import pyplot as plt 
import matplotlib.pylab as plt
import matplotlib.path as mpath
from matplotlib.pyplot import figure
import numpy as np
import cartopy.feature as cfeature
from matplotlib.cm import get_cmap
import pyproj
import utm
import pandas as pd 
import traceback 
from pathlib import Path  
import os  
import numpy.ma as ma
import netCDF4 as nc
from netCDF4 import Dataset
import glob
import datetime as dt
import metpy.calc as mpcalc
from metpy.units import units
from metpy.cbook import get_test_data
from metpy.io import metar
from metpy.plots.declarative import (BarbPlot, ContourPlot, FilledContourPlot, MapPanel,
                                     PanelContainer, PlotObs)
from netCDF4 import date2num,num2date
import wrf
from wrf import (getvar, interplevel, to_np, latlon_coords, get_cartopy,
                 cartopy_xlim, cartopy_ylim)
from wrf import getvar, interplevel, to_np, get_basemap, latlon_coords
from wrf import getvar, xy, interp2dxy
from wrf import to_np, getvar, CoordPair, vertcross
from matplotlib.cm import get_cmap
from matplotlib.ticker import (NullFormatter, ScalarFormatter)

In [2]:
#CURRENT DATA
cur_wrfout6 = Dataset('CUR_wrfout_d03_2018-03-25_06_00_00')
cur_wrfout10 = Dataset('CUR_wrfout_d03_2018-03-25_10_00_00')
cur_wrfout11= Dataset('CUR_wrfout_d03_2018-03-25_11_00_00')
cur_wrfout12 = Dataset('CUR_wrfout_d03_2018-03-25_12_00_00')
cur_wrfout13= Dataset('CUR_wrfout_d03_2018-03-25_13_00_00')

#SST DATA
sst_wrfout6 = Dataset('SST_wrfout_d03_2018-03-25_06_00_00')
sst_wrfout10 = Dataset('SST_wrfout_d03_2018-03-25_10_00_00')
sst_wrfout11 = Dataset('SST_wrfout_d03_2018-03-25_11_00_00')
sst_wrfout12 = Dataset('SST_wrfout_d03_2018-03-25_12_00_00')
sst_wrfout13 = Dataset('SST_wrfout_d03_2018-03-25_13_00_00')

In [3]:
#Get PBLH
#Not staggered
cur_pblh_10 = getvar(cur_wrfout10, "PBLH")
cur_pblh_11 = getvar(cur_wrfout11, "PBLH")
cur_pblh_12 = getvar(cur_wrfout12, "PBLH")
cur_pblh_13 = getvar(cur_wrfout13, "PBLH")

sst_pblh_10 = getvar(sst_wrfout10, "PBLH")
sst_pblh_11 = getvar(sst_wrfout11, "PBLH")
sst_pblh_12 = getvar(sst_wrfout12, "PBLH")
sst_pblh_13 = getvar(sst_wrfout13, "PBLH")

#Get U
cur_U_10 = getvar(cur_wrfout10, "U")
cur_U_11 = getvar(cur_wrfout11, "U")
cur_U_12 = getvar(cur_wrfout12, "U")
cur_U_13 = getvar(cur_wrfout13, "U")

sst_U_10 = getvar(sst_wrfout10, "U")
sst_U_11 = getvar(sst_wrfout11, "U")
sst_U_12 = getvar(sst_wrfout12, "U")
sst_U_13 = getvar(sst_wrfout13, "U")

cur_V_10 = getvar(cur_wrfout10, "V")
cur_V_11 = getvar(cur_wrfout11, "V")
cur_V_12 = getvar(cur_wrfout12, "V")
cur_V_13 = getvar(cur_wrfout13, "V")

sst_V_10 = getvar(sst_wrfout10, "V")
sst_V_11 = getvar(sst_wrfout11, "V")
sst_V_12 = getvar(sst_wrfout12, "V")
sst_V_13 = getvar(sst_wrfout13, "V")

In [4]:
cur_z10 = getvar(cur_wrfout10, "z")
cur_z11 = getvar(cur_wrfout11, "z")
cur_z12 = getvar(cur_wrfout12, "z")
cur_z13 = getvar(cur_wrfout13, "z")

sst_z10 = getvar(sst_wrfout10, "z")
sst_z11 = getvar(sst_wrfout11, "z")
sst_z12 = getvar(sst_wrfout12, "z")
sst_z13 = getvar(sst_wrfout13, "z")

z_sst_south = (sst_z10 + sst_z11 +sst_z12)/3
z_cur_south = (cur_z10 + cur_z11 +cur_z12)/3
z_sst_north = (sst_z11 + sst_z12 +sst_z13)/3
z_cur_north = (cur_z11 + sst_z12 +sst_z13)/3

print('goal shape: ')
print(z_sst_south .shape)

goal shape: 
(99, 474, 639)


In [5]:
cur_uN = (cur_U_11 + cur_U_12 +cur_U_13)/3
cur_uS = (cur_U_10 + cur_U_11 +cur_U_12)/3
sst_uN = (sst_U_11 + sst_U_12 +sst_U_13)/3
sst_uS = (sst_U_10 + sst_U_11 +sst_U_12)/3

cur_vN = (cur_V_11 + cur_V_12 +cur_V_13)/3
cur_vS = (cur_V_10 + cur_V_11 +cur_V_12)/3
sst_vN = (sst_V_11 + sst_V_12 +sst_V_13)/3
sst_vS = (sst_V_10 + sst_V_11 +sst_V_12)/3

print(cur_uN.shape)


(99, 474, 640)


In [6]:
N_cur_pblh = (cur_pblh_11 + cur_pblh_12 +cur_pblh_13) /3
N_sst_pblh = (sst_pblh_11 + sst_pblh_12 +sst_pblh_13) /3
S_cur_pblh = (cur_pblh_10 + cur_pblh_11 +cur_pblh_12) /3
S_sst_pblh = (sst_pblh_10 + sst_pblh_11 +sst_pblh_12) /3

N_pblh_line_sst = wrf.interpline(field2d = N_sst_pblh, wrfin=cur_wrfout12, timeidx=0, stagger=None, projection=None, ll_point=None, pivot_point=None, angle=None, start_point=CoordPair(lat=28.5, lon=-89.1), end_point=CoordPair(lat=27.7, lon=-89.1), latlon=True, cache=None, meta=True)
S_pblh_line_sst = wrf.interpline(field2d = S_sst_pblh, wrfin=cur_wrfout12, timeidx=0, stagger=None, projection=None, ll_point=None, pivot_point=None, angle=None, start_point=CoordPair(lat=26.6, lon=-89.9), end_point=CoordPair(lat=26.0, lon=-89.9), latlon=True, cache=None, meta=True)
N_pblh_line_cur = wrf.interpline(field2d = N_cur_pblh, wrfin=cur_wrfout12, timeidx=0, stagger=None, projection=None, ll_point=None, pivot_point=None, angle=None, start_point=CoordPair(lat=28.5, lon=-89.1), end_point=CoordPair(lat=27.7, lon=-89.1), latlon=True, cache=None, meta=True)
S_pblh_line_cur = wrf.interpline(field2d = S_cur_pblh, wrfin=cur_wrfout12, timeidx=0, stagger=None, projection=None, ll_point=None, pivot_point=None, angle=None, start_point=CoordPair(lat=26.6, lon=-89.9), end_point=CoordPair(lat=26.0, lon=-89.9), latlon=True, cache=None, meta=True)


In [ ]:
#Get Absolute Vorticity
#wrf.g_vorticity.get_avo(wrfin=, timeidx=0, method='cat', squeeze=True, cache=None, meta=True, _key=None)
#Relative Vorticity 




In [ ]:
#CUR NORTH
z = z_cur_north
theta = N_speed_cur
pblh_line = N_pblh_line_cur
start_point = CoordPair(lat=28.5, lon=-89.1)
end_point = CoordPair(lat=27.7, lon=-89.1)

theta_cross = vertcross(theta, z, wrfin=cur_wrfout12, start_point=start_point,
                       end_point=end_point, latlon=True, meta=True)

fig = plt.figure(figsize=(12,6), dpi=200.)
ax = plt.axes()
coord_pairs = to_np(theta_cross.coords["xy_loc"])
x = np.arange(coord_pairs.shape[0])
y = pblh_line

theta_contours = ax.contourf(np.arange(coord_pairs.shape[0]),to_np(theta_cross["vertical"]),
                           to_np(theta_cross),np.arange(4.0, 10.0, 0.5),cmap=get_cmap("viridis"))
plt.colorbar(theta_contours, ax=ax,  label = 'm/s')
theta_contours_line = ax.contour(np.arange(coord_pairs.shape[0]),to_np(theta_cross["vertical"]),
                           to_np(theta_cross),np.arange(4.0, 10.0, 0.5), colors = 'black')



plt.clabel(theta_contours_line, inline=1, fontsize=12, fmt='%1.2f', colors = 'black')
plt.plot(x, y, color = 'red' )
ax.annotate('PBLH',xy=(100, 85), xycoords='figure points', fontsize=12, color = 'red')
x_ticks = np.arange(coord_pairs.shape[0])
x_labels = [pair.latlon_str(fmt="{:.2f}, {:.2f}")
            for pair in to_np(coord_pairs)]
ax.set_yticks(np.linspace(0, 5000, 20))
ax.set_ylim(0,2000) #900 max PBLH + 500m = 1400m + 200 m buffer = 1600m
ax.set_xticks(x_ticks[::5])
ax.set_xticklabels(x_labels[::5], rotation=45, fontsize=10)
ax.set_xlabel("Latitude/Longitude", fontsize=12)
ax.set_ylabel("Height (m)", fontsize=12)
plt.title("Vorticity Vertical Cross Section\nNorth Subdomain (SSTs + Currents)")
save_results_to = '/Users/Anna/Desktop/WRF/Thesis_Plots/'
plt.savefig(save_results_to + 'north_vorticityCross_cur.png', bbox_inches = 'tight',dpi = 150)
plt.show()